<a href="https://colab.research.google.com/github/componavt/sns4human/blob/main/src/vk/topics/zero_shot_topic_classification_ru.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🔍📚 Zero-Shot Multilingual Topic Classifier

🎯 **Goal**: Automatically assign one of 24 expert-defined topics to thousands of short social media posts — without any fine-tuning!

🤖 **Model**: [`joeddav/xlm-roberta-large-xnli`](https://huggingface.co/joeddav/xlm-roberta-large-xnli) (based on XLM-RoBERTa)

🗂 **Input**: CSV file with columns:
- `text` — the post content
- `topic` — the expert-assigned label

📈 **Output**:
- `classified_posts.csv` with predicted topic and confidence score
- Evaluation metrics: `accuracy`, `precision`, `recall`, `F1-score`

🧪 Ideal for: fast prototyping of topic classification in Russian and other languages using Hugging Face's zero-shot pipeline.

⚡️ Tip: Batch-processing enabled for efficient GPU utilization.

In [1]:
# topics are indicated by an expert for each social network post
filename = '512_posts_24_topics.csv'

In [ ]:
from transformers import pipeline
import pandas as pd
from tqdm import tqdm
from io import StringIO
import requests
from io import StringIO
import requests
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Load zero-shot classification pipeline with multilingual support
classifier = pipeline("zero-shot-classification", model="joeddav/xlm-roberta-large-xnli")

# Load texts with 'text' and 'topic' columns
response = requests.get(f'https://raw.githubusercontent.com/componavt/sns4human/refs/heads/main/data/vk/topics/{filename}')
df = pd.read_csv(StringIO(response.text), delimiter=',', encoding='utf-8')

# Remove unassigned topic posts
df = df[df['topic'].str.lower() != 'пусто'].copy()

print(f"Total posts after filtering: {len(df)}")

texts       = df['text'].tolist()
true_labels = df['topic'].tolist()

# List of 24-1 topic labels defined by the expert
topic_labels = [
    "Эпос_Калевала", "благоустройство", "вепсы", "война", "выставка",
    "день рождения и юбилей", "еда кухня", "карельский язык", "конкурс",
    "литература", "мероприятия проекты", "музей. экскурсия", "музыка",
    "образование", "поэзия", "праздник", "природа", "социальная сфера",
    "традиция", "фестиваль", "фольклор", "этнокультура", "язык" #, "пусто"
]

# Store results
results = []
predicted_labels = []

# Batch processing
batch_size = 8
for i in tqdm(range(0, len(texts), batch_size), desc="Classifying posts"):
    batch = texts[i:i+batch_size]
    outputs = classifier(batch, topic_labels, multi_label=False)
    if isinstance(outputs, dict):  # if only one item
        outputs = [outputs]
    for output in outputs:
        top_label = output['labels'][0]
        confidence = output['scores'][0]
        predicted_labels.append(top_label)
        results.append({
            "text": output['sequence'],
            "predicted_topic": top_label,
            "confidence": confidence
        })

# Save to CSV
result_df = pd.DataFrame(results)
result_df['true_topic'] = true_labels
result_df.to_csv("classified_posts.csv", index=False, encoding="utf-8")

# Evaluate accuracy
print("\nClassification Report (Zero-Shot vs Expert Labels):")
print(classification_report(true_labels, predicted_labels, zero_division=0))

# Confusion matrix visualization
conf_matrix = confusion_matrix(true_labels, predicted_labels, labels=topic_labels)
conf_matrix_normalized = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]

# Create a mask for zero values
mask = conf_matrix_normalized == 0

plt.figure(figsize=(14, 12))
sns.heatmap(conf_matrix_normalized, xticklabels=topic_labels, yticklabels=topic_labels,
            annot=True, fmt=".0%", cmap="Blues", cbar=True, square=True,
            mask=mask,                # Mask cells with zero value
            cbar_kws={"shrink": 0.5})
plt.title("Confusion Matrix (Normalized by True Labels)")
plt.xlabel("Predicted Topic")
plt.ylabel("True Topic")
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()